<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/magistral_gpt4_aai_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-openai -q
!pip install colab-env -q
!pip install mistralai -q
!pip install openai-agents -q
!pip install langchain-mistralai -q
!pip install pydantic -q
!pip install crewai -q
!pip install crewai_tools -q
!pip install crewai['tools'] -q

In [ ]:
import colab_env

## AGENT

In [3]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
from crewai.tools import tool

# Load environment variables from .env file
load_dotenv()

# --- 0. Set up Environment Variables ---
openai_api_key = os.environ.get("OPENAI_API_KEY")
mistral_api_key = os.environ.get("MISTRAL_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it.")
if not mistral_api_key:
    raise ValueError("MISTRAL_API_KEY environment variable not set. Please set it.")

print("API keys loaded successfully.")

# --- 1. Define Tools (Now properly wrapped with @tool) ---
# Pydantic models for internal documentation/clarity and for generating API schemas
class WeatherInput(BaseModel):
    location: str = Field(description="The geographical location, e.g., 'Paris, France'")
    time: str = Field(description="The specific time or date for the forecast, e.g., 'tomorrow 10 AM Local Time'")

class NotamInput(BaseModel):
    airspace: str = Field(description="The identifier for the airspace, e.g., 'LAX' or 'EHAM'")

class ComplianceInput(BaseModel):
    route_details: str = Field(description="Detailed description of the proposed flight route.")
    regulations: List[str] = Field(description="List of specific aviation regulations to check against.")

class FuelCalcInput(BaseModel):
    aircraft_type: str = Field(description="The type of aircraft. e.g., 'Boeing 747'")
    distance_km: float = Field(description="The flight distance in kilometers.")
    weather_conditions: str = Field(description="Brief description of weather conditions influencing fuel burn.")

@tool("Get Weather Forecast") # Decorate with @tool from crewai_tools.tool_decorator
def get_weather_forecast(location: str, time: str) -> str:
    """Fetches detailed weather forecast for a given location and time."""
    print(f" [Tool:get_weather_forecast] Fetching weather for {location} at {time}...")
    if "storm" in location.lower() or "bad weather" in time.lower():
        return "Severe thunderstorms with high winds and low visibility expected. Instrument Flight Rules (IFR) conditions."
    return "Clear skies, light winds, good visibility. Visual Flight Rules (VFR) conditions."

@tool("Get NOTAMs") # Decorate with @tool from crewai_tools.tool_decorator
def get_notams(airspace: str) -> str:
    """Retrieves Notices to Airmen (NOTAMs) for a specified airspace."""
    print(f" [Tool:get_notams] Retrieving NOTAMS for {airspace}...")
    if "LAX" in airspace.upper():
        return "NOTAM: LAX Runway 25L closed for maintenance until 2025-07-15. Expect delays."
    if "LONDON HEATHROW" in airspace.upper() or "EGLL" in airspace.upper():
        return "NOTAM: EGLL ATC system experiencing minor disruptions, expect minor flow control."
    return "No significant NOTAMS for this airspace at this time."

@tool("Check Regulatory Compliance") # Decorate with @tool from crewai_tools.tool_decorator
def check_regulatory_compliance(route_details: str, regulations: List[str]) -> str:
    """Checks a proposed route against aviation regulations and provides detailed reasoning."""
    print(f" [Tool:check_regulatory_compliance] Checking regulatory compliance for route: '{route_details}' against regulations: {', '.join(regulations)}...")
    compliance_issues = []
    reasoning_steps = []

    if "restricted airspace" in route_details.lower() and not "special clearance" in route_details.lower():
        compliance_issues.append("Violation: Route enters restricted airspace without explicit clearance.")
        reasoning_steps.append("Identified 'restricted airspace' in route details. No mention of 'special clearance'.")

    current_weather_at_dest = get_weather_forecast("destination", "now") # Hypothetical call
    if "ICAO Annex 2" in regulations and "VFR" in route_details and "IFR" in current_weather_at_dest.lower():
        compliance_issues.append("Violation: Attempting Visual Flight Rules (VFR) in anticipated Instrument Flight Rules (IFR) conditions at destination.")
        reasoning_steps.append("Checked ICAO Annex 2. VFR flights require specific weather minima. Destination weather indicates IFR.")

    if not compliance_issues:
        return f"Compliance Check: PASSED. All specified regulations ({', '.join(regulations)}) appear to be met for the route '{route_details}'."
    else:
        return f"Compliance Check: FAILED.\nIssues: {'; '.join(compliance_issues)}\nDetailed Reasoning: {'; '.join(reasoning_steps)}"

@tool("Calculate Fuel Burn") # Decorate with @tool from crewai_tools.tool_decorator
def calculate_fuel_burn(aircraft_type: str, distance_km: float, weather_conditions: str) -> str:
    """Calculates estimated fuel burn for a given aircraft type, distance and weather."""
    print(f" [Tool:calculate_fuel_burn] Calculating fuel burn for {aircraft_type} over {distance_km:.0f}km with '{weather_conditions}'...")
    base_fuel_rate_per_km = 0.05
    if "Boeing 787" in aircraft_type:
        base_fuel_rate_per_km = 0.08

    fuel_burn = distance_km * base_fuel_rate_per_km

    if "high winds" in weather_conditions.lower() or "thunderstorms" in weather_conditions.lower():
        fuel_burn *= 1.15 # 15% increase for adverse weather

    return f"Estimated fuel burn: {fuel_burn:.2f} liters (adjusted for adverse weather)." if "high winds" in weather_conditions.lower() or "thunderstorms" in weather_conditions.lower() else f"Estimated fuel burn: {fuel_burn:.2f} liters."


# --- 2. Configure LLMs for CrewAI ---
# Instantiate OpenAI LLM for Route Optimization
openai_llm = ChatOpenAI(model_name="gpt-4o", api_key=openai_api_key)
print("OpenAI LLM (gpt-4o) configured for CrewAI.")

# Instantiate Mistral LLM for Regulatory & Risk
#model="magistral-medium-latest", # This agent doesn't use built-in connectors, can keep this model
#mistral_llm = ChatMistralAI(model="mistral/mistral-large-latest", api_key=mistral_api_key)
#print("Mistral LLM (mistral/mistral-large-latest) configured for CrewAI.")

mistral_llm = ChatMistralAI(model="mistral/magistral-medium-latest", api_key=mistral_api_key)
print("Mistral LLM (mistral/magistral-medium-latest) configured for CrewAI.")


# --- 3. Define Agents ---
route_optimizer_agent = Agent(
    role='Expert Flight Route Optimizer',
    goal='Propose the most efficient and safe flight path considering weather and fuel.',
    backstory=(
        "You are a seasoned aeronautical engineer and a master of route planning."
        "You analyze weather patterns, aircraft performance, and historical data"
        "to craft the optimal flight path for any given journey."
    ),
    llm=openai_llm, # Assign the OpenAI LLM to this agent
    tools=[get_weather_forecast, calculate_fuel_burn], # Pass the decorated functions
    verbose=False,
    allow_delegation=False
)
print("RouteOptimizerAgent defined with GPT-4o.")

regulatory_risk_agent = Agent(
    role='Aviation Regulatory and Risk Analyst',
    goal='Meticulously review proposed flight routes for compliance and identify potential risks.',
    backstory=(
        "You are a meticulous aviation law expert and a sharp-eyed risk assessor."
        "Your expertise lies in dissecting flight plans against a myriad of regulations"
        "and identifying any potential hazards or non-compliance issues."
    ),
    llm=mistral_llm, # Assign the Mistral LLM to this agent
    tools=[get_notams, check_regulatory_compliance], # Pass the decorated functions
    verbose=False,
    allow_delegation=False
)
print("RegulatoryRiskAgent defined with Mistral Large.")


# --- 4. Define Tasks ---
optimize_route_task = Task(
    description=(
        f"Propose the most efficient and safe flight route from {{origin}} to {{destination}} "
        f"for a {{aircraft_type}} aircraft. "
        "Consider current weather at both locations and estimate fuel burn. "
        "The output must include:\n"
        "1. Proposed Route Details (e.g., waypoints, altitude).\n"
        "2. Detailed Reasoning for the route, altitude, and fuel estimation.\n"
        "3. Estimated Fuel Burn in liters.\n"
        "4. Estimated Time of Arrival (ETA)."
    ),
    expected_output="A comprehensive flight route proposal including route details, reasoning, estimated fuel burn, and ETA.",
    agent=route_optimizer_agent,
    output_file='initial_route_proposal.md'
)
print("OptimizeRoute task defined.")

check_compliance_task = Task(
    description=(
        "Review the proposed flight route for full compliance with ICAO Annex 2 "
        "and national airspace regulations. Assess all potential risks, considering "
        "current NOTAMs for {{origin}} and {{destination}}."
        "Provide a transparent, step-by-step justification for every finding. "
        "If the route is compliant, state 'Compliance Check: PASSED'. "
        "If not, detail all 'Compliance Check: FAILED' issues and provide detailed reasoning."
    ),
    expected_output="A detailed compliance and risk assessment report. Clearly state PASSED or FAILED with justifications.",
    agent=regulatory_risk_agent,
    context=[optimize_route_task],
    output_file='final_compliance_report.md'
)
print("CheckCompliance task defined.")


# --- 5. Assemble and Run the Crew ---
flight_planning_crew = Crew(
    agents=[route_optimizer_agent, regulatory_risk_agent],
    tasks=[optimize_route_task, check_compliance_task],
    process=Process.sequential,
    verbose=False
)

print('\n')
print("Flight Planning Crew assembled.")

# --- Main execution block ---
if __name__ == "__main__":
    from datetime import datetime
    current_time = datetime.now().strftime("%A, %B %d, %Y at %I:%M:%S %p %Z")
    print(f"\n--- Initiating Flight Planning Workflow ({current_time}) ---")

    flight_inputs = {
        'origin': "Montreal, QC, Canada",
        'destination': "London, UK",
        'aircraft_type': "Boeing 787 Dreamliner"
    }

    print(f"\nPlanning flight from {flight_inputs['origin']} to {flight_inputs['destination']} "
          f"with {flight_inputs['aircraft_type']}.\n")

    final_result = flight_planning_crew.kickoff(inputs=flight_inputs)

    print("\n--- Flight Planning Workflow Completed ---")
    print("\nFinal Flight Planning Report:\n")
    print(final_result)

API keys loaded successfully.
OpenAI LLM (gpt-4o) configured for CrewAI.
Mistral LLM (mistral/magistral-medium-latest) configured for CrewAI.
RouteOptimizerAgent defined with GPT-4o.
RegulatoryRiskAgent defined with Mistral Large.
OptimizeRoute task defined.
CheckCompliance task defined.


Flight Planning Crew assembled.

--- Initiating Flight Planning Workflow (Wednesday, July 02, 2025 at 03:03:14 PM ) ---

Planning flight from Montreal, QC, Canada to London, UK with Boeing 787 Dreamliner.

 [Tool:get_weather_forecast] Fetching weather for Montreal, QC, Canada at current...
 [Tool:get_weather_forecast] Fetching weather for London, UK at current...
 [Tool:calculate_fuel_burn] Calculating fuel burn for Boeing 787 Dreamliner over 5226km with 'clear'...

--- Flight Planning Workflow Completed ---

Final Flight Planning Report:

Detailed Compliance and Risk Assessment Report

Route Compliance:
The proposed route from Montreal to London via North Atlantic Tracks (NATs) is compliant with ICA